This notebook assists with processing raw output seismograms generated by the AxiSEM3D software (https://github.com/AxiSEMunity/AxiSEM3D). It assumes that you have followed the AxiSEM3D file structure; in cell 2 you can point the notebook to the relevant directories.

In [1]:
import numpy as np
import yaml
import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal, fft
from subprocess import run
import re
import pandas as pd
import obspy
import obspy.signal.freqattributes as freqatt
from obspy.core.util.attribdict import AttribDict

In [54]:
### establish paths and variables ###
# location of AxiSEM3D master directory
axisem_loc = "~/Documents/Synthetics/axisem3d_root/AxiSEM3D"

# this notebook assumes that the simulation input, output directories etc are located in 
# run_dir = f'{axisem_loc}/{model_name}/simu{run_dims}/{run_name}'
# adjust file paths as necessary for your setup

model_name = 'LayeredCore_4Hz'
run_name = '2DVQC400'
run_dims = '3D'

run_dir = f'{axisem_loc}/{model_name}/simu{run_dims}/{run_name}'

# master directory where processed seismic traces will be stored
working_dir = "~/Documents/Enceladus/3D"

# choose which station file (will affect output path)
station_dims = '1D'
station_dict = {'1D': 'stations.txt',
               '3D': '3Dstations.txt'}

# create directory to store processed seismic data
output_dir = f'{working_dir}/{model_name}/simu{station_dims}/{run_name}'
run(['mkdir', '-p', output_dir])

CompletedProcess(args=['mkdir', '-p', '/Users/kd16230/Documents/Enceladus/3D_modelling/LayeredCore_4Hz/simu1D/2DVQC400'], returncode=0)

In [56]:
# read event location
run_dir = f'{axisem_loc}/{model_name}/simu{run_dims}/{run_name}'

sta_lon = {}
sta_lat = {}

with open(f'{run_dir}/input/inparam.source.yaml', 'r') as file:
    source_yaml = yaml.load(file, Loader=yaml.FullLoader)
# request named source from the simulation e.g. SOUTH_POLAR_MRR, SOUTH_POLAR_NORMAL
loc_leaf = source_yaml['list_of_sources'][0]['South_Polar_MRR']['location']
event_latlon = loc_leaf['latitude_longitude']
event_depth = loc_leaf['depth']

# read list of stations
with open(f'{run_dir}/input/{station_dict[station_dims]}','r') as file:
    for line in file:
        try:
            p = line.split()
            float(p[0])
            sta_lat[str(p[0])]=p[2]
            sta_lon[str(p[0])]=p[3]
        except:
            pass

In [58]:
time = np.loadtxt(f'{run_dir}/output/stations/Enceladus_stations/data_time.ascii')

#read 3-component data for every station
disp = {'R':{},
        'T':{},
        'Z':{}}
for station in sta_lat.keys():
    for ich, ch in enumerate('RTZ'):
        disp[ch][station] = np.array(np.loadtxt(f'{run_dir}/output/stations/Enceladus_stations/II.{station}.ascii')[:,ich])

In [59]:
### construct filter ###

sps = 1/float(time[1]-time[0]) # samples per second
order = 4             # Order of the filter (corners)
filter_type = 'bandpass'  # Filter type (e.g., 'highpass', 'lowpass', 'bandpass')
band_cutoff = [3,4]       # Critical frequency or frequencies for filter in Hz (use [min_freq, max_freq] for 'bandpass')

# Construct Butterworth filter
sos = signal.butter(order, band_cutoff, btype=filter_type, fs=sps, output='sos')

In [60]:
### process + write data ###

# for each station:
for station in sta_lat.keys():
    s2write = station
    
    # for each component:
    for component in disp.keys():
        
        # write processed unfiltered data
        amp2 = obspy.Trace(data = disp[component][station].T)
        amp2.detrend('simple')
        amp2.detrend('demean')
        amp2.taper(0.05,type='hann')
        amp2.stats['sampling_rate'] = sps
        amp2.stats['station'] = s2write
        amp2.write(f'{output_dir}/{s2write}{component}raw.sac')

        # write processed filtered data
        f_data = signal.sosfiltfilt(sos, amp2, padtype='odd')
        f_data2 = obspy.Trace(data = np.array(f_data))
        f_data2.stats['sampling_rate'] = sps
        f_data2.stats['station'] = s2write
        f_data2.write(f"{output_dir}/{s2write}{component}filt_{str(band_cutoff[0]).replace('.','')}_{str(band_cutoff[1]).replace('.','')}.sac")
        
        # write processed unfiltered acceleration data
        amp2.differentiate()
        amp2.differentiate()
        amp2.write(f"{output_dir}/{s2write}{component}raw_acc.sac")
        
        # write processed filtered acceleration data
        f_data3 = signal.sosfiltfilt(sos,amp2,padtype='odd')
        f_data4 = obspy.Trace(data = np.array(f_data3))
        f_data4.stats['sampling_rate'] = sps
        f_data4.stats['station'] = s2write
        f_data4.write(f"{output_dir}/{s2write}{component}filt_{str(band_cutoff[0]).replace('.','')}_{str(band_cutoff[1]).replace('.','')}_acc.sac")